In [1]:
import pennylane as qml
import numpy as np

In [2]:
#A.4.1
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)

def multisol_oracle_matrix(combos):
    my_array = np.identity(2 ** n_bits)
    
    indices = [np.ravel_multi_index(combo, [2]*len(combo)) for combo in combos]
    for i in range(len(combos)):
        my_array[indices[i], indices[i]] = -1
    return my_array
    pass

@qml.qnode(dev)
def multisol_pair_circuit(x_tilde, combos):
    for i in range(n_bits-1): # Initialize x_tilde part of state
        if x_tilde[i] == 1:
            qml.PauliX(wires=i)
    #Apply circuit
    qml.Hadamard(wires = n_bits-1)
    qml.QubitUnitary(multisol_oracle_matrix(combos), wires = list(range(n_bits)))
    qml.Hadamard(wires = n_bits-1)
    return qml.probs(wires=n_bits-1)

In [ ]:
#A.4.2
def parity_checker(combos):
    parity = 0
    counter = 0
    x_tilde_strs = [np.binary_repr(n, n_bits-1) for n in range(2**(n_bits-1))]
    x_tildes = [[int(s) for s in x_tilde_str] for x_tilde_str in x_tilde_strs]
    
    for x_tilde in x_tildes:
        if np.isclose(multisol_pair_circuit(x_tilde, combos)[1], 1):
            counter += 1
            
    parity = counter % 2
    return parity